In [1]:
import pandas as pd
import pickle

data = pd.read_csv('test.csv')
data

,ID,TITLE,ABSTRACT
0,20973,Closed-form Marginal Likelihood in Gamma-Poiss...,We present novel understandings of the Gamma...
1,20974,Laboratory mid-IR spectra of equilibrated and ...,Meteorites contain minerals from Solar Syste...
2,20975,Case For Static AMSDU Aggregation in WLANs,Frame aggregation is a mechanism by which mu...
3,20976,The $Gaia$-ESO Survey: the inner disk intermed...,Milky Way open clusters are very diverse in ...
4,20977,Witness-Functions versus Interpretation-Functi...,Proving that a cryptographic protocol is cor...
...,...,...,...
8984,29957,Supporting mixed-datatype matrix multiplicatio...,We approach the problem of implementing mixe...
8985,29958,An axiomatic basis for Blackwell optimality,In the theory of Markov decision processes (...
8986,29959,GeneVis - An interactive visualization tool fo...,GeneVis is a web-based tool to visualize com...
8987,29960,Quantifying the causal effect of speed cameras...,This paper quantifies the effect of speed ca...


In [2]:
from gensim import matutils, models
import scipy.sparse

In [3]:
abstract=data['ABSTRACT']

In [4]:
abstract

0         We present novel understandings of the Gamma...
1         Meteorites contain minerals from Solar Syste...
2         Frame aggregation is a mechanism by which mu...
3         Milky Way open clusters are very diverse in ...
4         Proving that a cryptographic protocol is cor...
                              ...                        
8984      We approach the problem of implementing mixe...
8985      In the theory of Markov decision processes (...
8986      GeneVis is a web-based tool to visualize com...
8987      This paper quantifies the effect of speed ca...
8988      We show that the vertices and edges of a $d$...
Name: ABSTRACT, Length: 8989, dtype: object

In [9]:
import re
data['ABSTRACT'] = data['ABSTRACT'].apply(lambda x: re.sub(r'[^\w\s]','', str(x)))
data['ABSTRACT'] = data['ABSTRACT'].apply(lambda x: re.sub(r'[0-9]','', str(x)))
data['ABSTRACT'] = data['ABSTRACT'].apply(lambda x: re.sub('\n','', str(x)))
data['ABSTRACT'] = data['ABSTRACT'].apply(lambda x: re.sub('\t','', str(x)))
data['ABSTRACT'] = data['ABSTRACT'].str.lower()

In [10]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from collections import Counter

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data.ABSTRACT)
dt = pd.DataFrame(data_cv.toarray()).iloc[:len(data)]  
dt.columns = cv.get_feature_names()
dt.index = data.TITLE
document_term_matrix = dt.T
document_term_matrix.columns = ['ABSTRACT '+str(i+1) for i in range(len(data))]
document_term_matrix['total_count'] = document_term_matrix.sum(axis=1)

document_term_matrix = document_term_matrix.sort_values(by ='total_count',ascending=False)[:2000] 

#print(document_term_matrix.drop(columns=['total_count']).head(10))
most_common = document_term_matrix.index
document_term_matrix = document_term_matrix.drop(columns=['total_count'])
dtm1 = pd.DataFrame(document_term_matrix)

c:\PYTHON39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [12]:
dtm1

,ABSTRACT 1,ABSTRACT 2,ABSTRACT 3,ABSTRACT 4,ABSTRACT 5,ABSTRACT 6,ABSTRACT 7,ABSTRACT 8,ABSTRACT 9,ABSTRACT 10,...,ABSTRACT 8980,ABSTRACT 8981,ABSTRACT 8982,ABSTRACT 8983,ABSTRACT 8984,ABSTRACT 8985,ABSTRACT 8986,ABSTRACT 8987,ABSTRACT 8988,ABSTRACT 8989
model,2,0,1,0,0,2,1,0,0,0,...,0,2,0,0,0,0,0,0,0,0
data,1,0,0,1,0,1,0,5,0,0,...,0,0,0,0,0,0,0,3,0,0
using,0,0,0,1,0,0,0,0,1,0,...,0,0,1,1,0,0,0,1,1,0
paper,0,0,0,0,1,1,0,0,1,0,...,0,0,1,0,1,0,1,0,1,0
results,0,0,0,0,0,0,2,1,1,1,...,0,0,0,0,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
industry,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
arbitrarily,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
rnns,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
interplay,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
import scipy.sparse
from gensim import matutils, models
sparse_counts = scipy.sparse.csr_matrix(dtm1)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [15]:
id2word = {}
keys = []
for i in range(len(dtm1)):
  j = dtm1.index.get_loc(dtm1.index[i])
  keys.append(j)

for key, val in zip(keys, dtm1.index):
  id2word[key] = val

id2word

{0: 'model',
 1: 'data',
 2: 'using',
 3: 'paper',
 4: 'results',
 5: 'method',
 6: 'problem',
 7: 'learning',
 8: 'based',
 9: 'models',
 10: 'new',
 11: 'approach',
 12: 'study',
 13: 'network',
 14: 'time',
 15: 'algorithm',
 16: 'number',
 17: 'used',
 18: 'methods',
 19: 'proposed',
 20: 'systems',
 21: 'networks',
 22: 'different',
 23: 'propose',
 24: 'present',
 25: 'analysis',
 26: 'use',
 27: 'function',
 28: 'work',
 29: 'performance',
 30: 'set',
 31: 'information',
 32: 'space',
 33: 'large',
 34: 'neural',
 35: 'algorithms',
 36: 'order',
 37: 'provide',
 38: 'structure',
 39: 'case',
 40: 'given',
 41: 'field',
 42: 'energy',
 43: 'state',
 44: 'properties',
 45: 'deep',
 46: 'theory',
 47: 'framework',
 48: 'high',
 49: 'distribution',
 50: 'linear',
 51: 'demonstrate',
 52: 'process',
 53: 'problems',
 54: 'prove',
 55: 'result',
 56: 'features',
 57: 'parameters',
 58: 'novel',
 59: 'general',
 60: 'rate',
 61: 'training',
 62: 'quantum',
 63: 'functions',
 64: 'appli

In [18]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.016*"data" + 0.011*"model" + 0.009*"learning" + 0.008*"using" + 0.008*"network" + 0.007*"models" + 0.007*"paper" + 0.007*"approach" + 0.007*"based" + 0.007*"method"'),
 (1,
  '0.007*"study" + 0.007*"model" + 0.007*"results" + 0.006*"problem" + 0.005*"theory" + 0.005*"function" + 0.005*"paper" + 0.005*"field" + 0.005*"number" + 0.005*"using"')]

In [16]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.012*"problem" + 0.009*"paper" + 0.008*"number" + 0.008*"function" + 0.008*"prove" + 0.007*"results" + 0.007*"space" + 0.007*"set" + 0.007*"study" + 0.006*"case"'),
 (1,
  '0.011*"model" + 0.009*"energy" + 0.007*"phase" + 0.007*"field" + 0.007*"using" + 0.007*"quantum" + 0.006*"magnetic" + 0.006*"study" + 0.006*"results" + 0.005*"dynamics"'),
 (2,
  '0.017*"data" + 0.012*"model" + 0.011*"learning" + 0.009*"network" + 0.008*"models" + 0.008*"using" + 0.008*"approach" + 0.008*"paper" + 0.007*"based" + 0.007*"method"')]

In [17]:


ldana = models.LdaModel(corpus=corpus, num_topics=4, id2word=id2word, passes=10)
ldana.print_topics()

[(0,
  '0.015*"data" + 0.013*"learning" + 0.011*"network" + 0.010*"model" + 0.008*"networks" + 0.008*"paper" + 0.008*"approach" + 0.008*"using" + 0.007*"based" + 0.007*"models"'),
 (1,
  '0.018*"data" + 0.016*"model" + 0.009*"models" + 0.009*"distribution" + 0.008*"using" + 0.008*"method" + 0.007*"results" + 0.006*"used" + 0.006*"analysis" + 0.006*"estimation"'),
 (2,
  '0.011*"energy" + 0.011*"model" + 0.010*"phase" + 0.010*"quantum" + 0.009*"field" + 0.008*"dynamics" + 0.008*"magnetic" + 0.007*"state" + 0.007*"study" + 0.006*"states"'),
 (3,
  '0.015*"problem" + 0.010*"paper" + 0.009*"prove" + 0.008*"number" + 0.008*"set" + 0.007*"function" + 0.007*"space" + 0.007*"results" + 0.007*"algorithm" + 0.007*"functions"')]

In [18]:

ldana = models.LdaModel(corpus=corpus, num_topics=4, id2word=id2word, passes=80)
ldana.print_topics()

[(0,
  '0.011*"prove" + 0.010*"space" + 0.010*"paper" + 0.009*"graph" + 0.008*"study" + 0.008*"theory" + 0.008*"set" + 0.008*"group" + 0.008*"result" + 0.008*"results"'),
 (1,
  '0.010*"model" + 0.009*"energy" + 0.008*"phase" + 0.007*"using" + 0.007*"field" + 0.006*"quantum" + 0.006*"magnetic" + 0.006*"results" + 0.006*"study" + 0.005*"state"'),
 (2,
  '0.015*"algorithm" + 0.014*"problem" + 0.013*"method" + 0.012*"model" + 0.009*"time" + 0.008*"proposed" + 0.008*"paper" + 0.007*"based" + 0.007*"function" + 0.007*"methods"'),
 (3,
  '0.018*"data" + 0.014*"learning" + 0.012*"network" + 0.010*"model" + 0.010*"networks" + 0.008*"using" + 0.008*"paper" + 0.007*"models" + 0.007*"neural" + 0.007*"approach"')]

In [19]:
df2=data

In [20]:
#function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [24]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
data_nouns = pd.DataFrame(data.ABSTRACT.apply(nouns))
data_nouns

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,ABSTRACT
0,understandings gammapoisson gap model matrix f...
1,meteorites minerals system asteroids different...
2,frame aggregation mechanism frames transmissio...
3,way clusters terms age chemicalcomposition pro...
4,protocol secrecy taskone strategies goal secur...
...,...
8984,problem support matrix multiplication gemm ope...
8985,theory decision blackwell policyis policy disc...
8986,genevis tool data sets differentdisciplines fi...
8987,paper effect speed cameras road traffic doubly...


In [27]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# add_stop_words = ['aa','flightled','jetblue','need','im','dont','just','guys']
# stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer()
data_cvn = cvn.fit_transform(data_nouns.ABSTRACT)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

c:\PYTHON39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,__,__maser,_au,_f_,_f_left,_i,_ia,_ide,_isubgroup,_knot,...,ésik,été,être,öffentliches,özgün,übergravity,ünlü,čech,świątkowskia,ševera
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8984,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8985,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8986,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8987,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
tdm3 = data_dtmn.transpose()
tdm3.head(10)

,0,1,2,3,4,5,6,7,8,9,...,8979,8980,8981,8982,8983,8984,8985,8986,8987,8988
__,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
__maser,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_au,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_f_,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_f_left,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_i,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_ia,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_ide,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_isubgroup,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_knot,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(tdm3))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [30]:
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.013*"data" + 0.010*"model" + 0.008*"paper" + 0.007*"network" + 0.007*"problem" + 0.006*"models" + 0.006*"method" + 0.006*"approach" + 0.006*"results" + 0.006*"methods"'),
 (1,
  '0.005*"model" + 0.005*"results" + 0.005*"field" + 0.004*"system" + 0.004*"energy" + 0.004*"theory" + 0.004*"space" + 0.003*"phase" + 0.003*"paper" + 0.003*"properties"')]

In [31]:
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()


[(0,
  '0.006*"theory" + 0.005*"paper" + 0.005*"space" + 0.005*"results" + 0.004*"number" + 0.004*"problem" + 0.004*"result" + 0.004*"case" + 0.004*"group" + 0.004*"quantum"'),
 (1,
  '0.014*"data" + 0.010*"model" + 0.008*"paper" + 0.008*"problem" + 0.008*"network" + 0.007*"method" + 0.007*"models" + 0.007*"approach" + 0.006*"results" + 0.006*"methods"'),
 (2,
  '0.007*"model" + 0.006*"energy" + 0.005*"field" + 0.004*"system" + 0.004*"results" + 0.004*"data" + 0.004*"mass" + 0.003*"time" + 0.003*"dynamics" + 0.003*"temperature"')]

In [32]:
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.008*"paper" + 0.008*"problem" + 0.007*"space" + 0.006*"number" + 0.006*"results" + 0.005*"case" + 0.005*"result" + 0.005*"function" + 0.005*"theory" + 0.005*"group"'),
 (1,
  '0.016*"data" + 0.010*"model" + 0.009*"paper" + 0.009*"network" + 0.008*"problem" + 0.007*"approach" + 0.007*"method" + 0.007*"networks" + 0.007*"models" + 0.007*"methods"'),
 (2,
  '0.006*"energy" + 0.006*"data" + 0.006*"mass" + 0.004*"model" + 0.004*"results" + 0.004*"observations" + 0.003*"system" + 0.003*"field" + 0.003*"galaxies" + 0.003*"time"'),
 (3,
  '0.012*"model" + 0.006*"phase" + 0.006*"state" + 0.006*"dynamics" + 0.005*"systems" + 0.005*"system" + 0.005*"results" + 0.005*"states" + 0.005*"models" + 0.005*"properties"')]

In [33]:
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [34]:
data_nouns_adj = pd.DataFrame(data.ABSTRACT.apply(nouns_adj))
data_nouns_adj

,ABSTRACT
0,novel understandings gammapoisson gap model ap...
1,meteorites minerals solar system asteroids dif...
2,frame aggregation mechanism multiple frames as...
3,milky way open clusters diverse terms age chem...
4,cryptographic protocol correct secrecy hard ta...
...,...
8984,problem mixeddatatype support thegeneral matri...
8985,theory markov decision blackwell optimal polic...
8986,genevis webbased tool complementary data sets ...
8987,paper effect speed cameras road traffic approx...


In [35]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english')
df_cv = cv.fit_transform(data_nouns_adj.ABSTRACT)
dt = pd.DataFrame(df_cv.toarray()).iloc[:len(data)]  
dt.columns = cv.get_feature_names()
dt.index = data_nouns_adj.index
document_term_matrix = dt.T
document_term_matrix.columns = ['Text '+str(i+1) for i in range(len(data_nouns_adj))]
document_term_matrix['total_count'] = document_term_matrix.sum(axis=1)

document_term_matrix = document_term_matrix.sort_values(by ='total_count',ascending=False)[:2000] 

# print(document_term_matrix.drop(columns=['total_count']).head(10))
most_common = document_term_matrix.index
document_term_matrix = document_term_matrix.drop(columns=['total_count'])
tdm4 = pd.DataFrame(document_term_matrix)

c:\PYTHON39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [36]:
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(tdm4))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cv.vocabulary_.items())

In [37]:
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.005*"acc" + 0.001*"adultsubjects" + 0.001*"adversarialquantum" + 0.000*"achievestateoftheart" + 0.000*"acidulation" + 0.000*"advancedligo" + 0.000*"agoaldirected" + 0.000*"acorrupted" + 0.000*"accompanying" + 0.000*"acentral"'),
 (1,
  '0.010*"__" + 0.009*"__maser" + 0.007*"_au" + 0.007*"_dcal" + 0.006*"_f_" + 0.005*"_f_left" + 0.005*"_i" + 0.005*"_ia" + 0.005*"_isubgroup" + 0.005*"_ide"')]

In [38]:
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.006*"__maser" + 0.006*"_dcal" + 0.006*"a_and" + 0.005*"a_b" + 0.005*"a_breo_asr" + 0.005*"a_rm" + 0.005*"a_tb_t" + 0.004*"_s" + 0.004*"_ia" + 0.004*"a_b_ldotsa_kb_kc"'),
 (1,
  '0.000*"againstadversarial" + 0.000*"adaptivevariant" + 0.000*"actuarial" + 0.000*"ahotspotunaware" + 0.000*"aggregated" + 0.000*"adiscriminator" + 0.000*"adaptiveamsdu" + 0.000*"agradient" + 0.000*"aforementionedassumption" + 0.000*"actionable"'),
 (2,
  '0.015*"__" + 0.011*"__maser" + 0.009*"_au" + 0.008*"_f_" + 0.008*"_ide" + 0.007*"_i" + 0.007*"_f_left" + 0.007*"_isubgroup" + 0.007*"_dcal" + 0.007*"_m_oplus"')]

In [39]:
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.017*"__" + 0.012*"__maser" + 0.009*"_ide" + 0.008*"_au" + 0.008*"_i" + 0.008*"_isubgroup" + 0.008*"_f_left" + 0.007*"_dcal" + 0.007*"_mathrmrig" + 0.007*"_n"'),
 (1,
  '0.008*"a_b" + 0.008*"__maser" + 0.008*"a_rm" + 0.007*"a_and" + 0.007*"aapproximation" + 0.007*"a_tb_t" + 0.005*"a_b_" + 0.005*"abeliansquares" + 0.005*"abipartite" + 0.005*"abiotic"'),
 (2,
  '0.000*"advective" + 0.000*"aggregated" + 0.000*"akahler" + 0.000*"acyberagent" + 0.000*"ag_biterminated" + 0.000*"acousticsensors" + 0.000*"aeronautics" + 0.000*"achievesorders" + 0.000*"accumulationlayer" + 0.000*"acquisitionand"'),
 (3,
  '0.011*"_f_" + 0.008*"_au" + 0.007*"_poperatornamespecmathbbf_pu" + 0.007*"_linfty" + 0.007*"_dcal" + 0.007*"_s" + 0.006*"a_a_dots" + 0.006*"a_breo_asr" + 0.006*"a_pclasses" + 0.006*"a_inftystructures"')]